Install the dependencies we are going to use. JuMP is a Julia mathematical optimization front end and problem builder, CSV parses .csv files, and GLPK is a free open source solver. (add these at the julia repl) with <;add JuMP>

In [4]:
using JuMP
using CSV
using GLPK

┌ Info: Precompiling GLPK [60bf3e95-4087-53dc-ae20-288a0d20c6a6]
└ @ Base loading.jl:1242


First we will parse the problem specification.

In [12]:
problem = CSV.read("problem.csv"; delim="\t", header=false)


7

Build the problem into a mathematical expression. Note that all data is stored as binary data.

In [19]:
model = Model(
    with_optimizer(GLPK.Optimizer, tm_lim = 60000, msg_lev = GLPK.OFF)
)
@variable(model, entries[1:9,1:9,1:9], Bin)

#only one number in each cell
foreach(1:9) do idxX
    foreach(1:9) do idxY
        @constraint(model, sum(entries[idxX,idxY,:]) == 1)
    end
end
# each row has one of each entry
foreach(1:9) do idxX
    foreach(1:9) do idxEntry 
        @constraint(model, sum(entries[idxX, :, idxEntry]) == 1)
    end
end
# each column has one of each entry
foreach(1:9) do idxY
    foreach(1:9) do idxEntry 
        @constraint(model, sum(entries[:, idxY, idxEntry]) == 1)
    end
end
# each block has one of each entry
foreach(1:3) do blockXidx
    foreach(1:3) do blockYidx
        foreach(1:9) do idxEntry
            rangeX = (blockXidx-1)*3+1:(blockXidx)*3
            rangeY = (blockYidx-1)*3+1:(blockYidx)*3
            @constraint(model, sum(entries[rangeX,rangeY,idxEntry]) == 1)
        end
    end
end
# set the values from the given problem
foreach(1:9) do idxX
    foreach(1:9) do idxY
        foreach(1:9) do idxEntry
            if problem[idxX, idxY] == idxEntry
                @constraint(model, entries[idxX,idxY,idxEntry] == 1)
            end
        end
    end
end

Solve the problem.

In [20]:
optimize!(model)

In [21]:
entryData = round.(value.(entries))
result = [findfirst(val == 1, entryData[idxX,idxY,:]) for idxX in 1:9, idxY in 1:9 ]

9×9×9 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.